## Load in packages and data

In [1]:
import numpy as np
import pandas as pd
# import geopandas as gpd
import matplotlib.pyplot as plt
import matsim_output_analysis as moa
import operator

In [3]:
df = pd.read_csv(r"200_sample_formatted.csv") 

## Calculating utility components

Travel components

In [4]:
# 1. Get the trips out 
df["selected plan trips"] = df.apply(lambda row: moa.group_legs_into_trips(row["selected plan activity_type_or_mode"], row["activities_indices"]), axis=1)
df["unselected plan (1) trips"] = df.apply(lambda row: moa.group_legs_into_trips(row["unselected plan (1) activity_type_or_mode"], row["activities_indices"]), axis=1)
df["unselected plan (2) trips"] = df.apply(lambda row: moa.group_legs_into_trips(row["unselected plan (2) activity_type_or_mode"], row["activities_indices"]), axis=1)
df["unselected plan (3) trips"] = df.apply(lambda row: moa.group_legs_into_trips(row["unselected plan (3) activity_type_or_mode"], row["activities_indices"]), axis=1)
df["unselected plan (4) trips"] = df.apply(lambda row: moa.group_legs_into_trips(row["unselected plan (4) activity_type_or_mode"], row["activities_indices"]), axis=1)

TypeError: can only concatenate str (not "int") to str

In [ ]:
# 1. Keep corresponding trip durations and distances
df["selected plan trips_duration"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["selected plan duration"], row["activities_indices"], "duration"), axis=1)
df["unselected plan (1) trips_duration"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (1) duration"], row["activities_indices"], "duration"), axis=1)
df["unselected plan (2) trips_duration"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (2) duration"], row["activities_indices"], "duration"), axis=1)
df["unselected plan (3) trips_duration"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (3) duration"], row["activities_indices"], "duration"), axis=1)
df["unselected plan (4) trips_duration"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (4) duration"], row["activities_indices"], "duration"), axis=1)

df["selected plan trips_distance"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["selected plan distance_travelled"], row["activities_indices"], "distance"), axis=1)
df["unselected plan (1) trips_distance"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (1) distance_travelled"], row["activities_indices"], "distance"), axis=1)
df["unselected plan (2) trips_distance"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (2) distance_travelled"], row["activities_indices"], "distance"), axis=1)
df["unselected plan (3) trips_distance"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (3) distance_travelled"], row["activities_indices"], "distance"), axis=1)
df["unselected plan (4) trips_distance"] = df.apply(lambda row: moa.group_legs_into_trips_d(row["unselected plan (4) distance_travelled"], row["activities_indices"], "distance"), axis=1)

In [ ]:
df["calculated_travel_utility"] = df.apply(lambda row: moa.calculate_travel_utility(row["selected plan trips"], row["selected plan trips_duration"], row["selected plan trips_distance"], row["subpopulation"]), axis=1)

Activity utility

look at overall times column. start time of activity i = end time of activity i-1 + sum of times between them
so

In [ ]:
# just for initial sense checking, see if the calcs work for "well behaved" agents, ie those who DO wraparound and have positive utility
df_wellbehaved = df[df["selected plan utility"]>0]
df_wellbehaved = df_wellbehaved[df_wellbehaved["wraparound"]==1]

In [ ]:
def get_activity_durations(activity_indices, all_durations, all_activities_trips):
    activity_start_times = []
    activity_end_times = []
    all_durations = pd.to_timedelta(all_durations, errors="coerce").seconds

    for i in range(len(activity_indices)):
        activity_end_times.append(all_durations[activity_indices[i]])
        if activity_indices[i]==0:
            tempStart = 0
        else:
            prevEndTime = all_durations[activity_indices[i-1]]
            x1 = activity_indices[i-1]+1
            x2 = activity_indices[i]
            tempStart = prevEndTime+sum(all_durations[x1:x2])  
        activity_start_times.append(tempStart)
    # print("start: "+str(activity_start_times))
    # print("end: "+str(activity_end_times))
    
    for i in range(1, len(activity_end_times)):
        if activity_end_times[i] < activity_end_times[i-1]:
            activity_end_times[i] += 24*3600  #account for wraparound
    # print("end new: "+str(activity_end_times))
   
    activity_durations = list(map(operator.sub, list(activity_end_times), list(activity_start_times)))
   
    if all_activities_trips[0] == all_activities_trips[-1]:
        activity_durations[0]=activity_durations[0]+((24*3600)-activity_start_times[-1]) #make it wrap around to midnight
        activity_durations.pop() #remove last activity duration as it has been included in the frist activity
    
    for k in range(len(activity_durations)):
        if activity_durations[k]== 0:
            activity_durations[k] = 1e-5
    
    return(activity_durations)     

# TODO: account for non-wraparound effect - HOW DOES THIS WORK
# TODO: when accounting for wraparound effect the final "end time" is basically ignored - is this ok?
# TODO: account for if someone arrives after departure -> what to do? check experienced plans?
# TODO: cutoff when not achieving full activity plan - up to 32 hours / journey longer than expected departure

In [ ]:
def get_activity_durations_terrible(activity_indices, all_durations):
    activity_start_times = []
    activity_end_times = []
    all_durations = pd.to_timedelta(all_durations, errors="coerce").seconds

    arrive_after_start = 0

    for i in range(len(activity_indices)):
        activity_end_times.append(all_durations[activity_indices[i]])
        if activity_indices[i]==0:
            tempStart = 0
        else:
            prevEndTime = all_durations[activity_indices[i-1]]
            x1 = activity_indices[i-1]+1
            x2 = activity_indices[i]
            tempStart = prevEndTime+sum(all_durations[x1:x2])  
        activity_start_times.append(tempStart)
    # print("start: "+str(activity_start_times))
    # print("end: "+str(activity_end_times))
    
    for i in range(1, len(activity_end_times)):
        if activity_end_times[i] < activity_end_times[i-1]:
            activity_end_times[i] += 24*3600  #account for wraparound
    # print("end new: "+str(activity_end_times))

    for i in range(len(activity_start_times)):
        if activity_start_times[i]>activity_end_times[i]:
            arrive_after_start += 1
    
    return(arrive_after_start)     
#run this across population and examine the experienced plans for any with an output > 0

In [ ]:
# df["b"] = df.apply(lambda row: get_activity_durations_terrible(row["activities_indices"], row["selected plan durations"]), axis=1)

In [ ]:
# z = ["home","car","work","car", "home"]
# # y = ["08:00:00", "00:23:30", "08:53:30", "01:21:15", "23:10:00"]
# x = [0,2,4]

In [ ]:
# a = get_activity_durations_terrible(x, y, z)
# print(a)

In [ ]:
def calculate_activity_utility(activities, durations):
    utilities = []
    # betaEarly = 0
    # betaLate = 0
    betaPerf = 10
    # betaWait = 0
    # betaShort = 0
    prio = 1
    durations = [a/3600 for a in durations]  #put it back into hours
    if activities[-1] == activities[0]:
        activities.pop() #remove the last activity for wraparound
    for i in range(len(activities)):
        # print(activities[i])
        # print(durations[i])
        STotal = 0
        match activities[i]:
            case "home":
                tMin = 1
                tTyp = 10
            case "work":
                tMin = 6
                tTyp = 9
            case "other":
                tMin = 1/6
                tTyp = 0.5
            case "shop": 
                tMin = 0.5
                tTyp = 0.5
            case "education":
                tMin = 6
                tTyp = 6
            case "visit":
                tMin = 0.5
                tTyp = 2
            case "medical":
                tMin = 0.5
                tTyp = 1
            case "business":
                tMin = 0.5
                tTyp = 1
            case "escort_home":
                tMin = 1/12
                tTyp = 1/12
            case "escort_work":
                tMin = 1/12
                tTyp = 1/12
            case "escort_business":
                tMin = 1/12
                tTyp = 1/12
            case "escort_education":
                tMin = 1/12
                tTyp = 1/12
            case "escort_other":
                tMin = 1/12
                tTyp = 1/12
            case "escort_shop":
                tMin = 1/12
                tTyp = 1/12      
            case _:
                print("dodgy activity type")
                tMin = np.nan
                tTyp = np.nan
        # print("tTyp: "+str(tTyp))
        t0 = tTyp*np.exp(-1/prio)
        # print("t0: "+str(t0))
        SDur = betaPerf*tTyp*np.log(durations[i]/t0)
        SWait = 0 #since betaWait is zero. Otherwise, will need to encode open/close times for activities to find out how long waiting       
        SLate = 0 #since betaLate is zero. Otherwise, will need to encode latest start times for activities to find out how late   
        SEarly = 0 #since betaEarly is zero. Otherwise, will need to encode earliest end times for activities to find out how early   
        SShort = 0 #since betaEarly is zero/undefined. Otherwise, will need to use shortesr durations for activities to find out if too short   
        STotal_temp = SDur + SWait + SLate + SEarly + SShort
        STotal += STotal_temp
        # print("STotal: "+str(STotal))
        utilities.append(STotal)
    return utilities
    

In [ ]:
def get_activities(activity_modes, activity_indices):
    activities = []
    for i in activity_indices:
        activities.append(activity_modes[i])
    return activities

In [ ]:
df_wellbehaved["activity_durations"] = df_wellbehaved.apply(lambda row: get_activity_durations(row["activities_indices"], row["selected plan duration"], row["selected plan activity_type_or_mode"]), axis=1)

In [ ]:
df_wellbehaved["selected plan activities"] = df_wellbehaved.apply(lambda row: get_activities(row["selected plan activity_type_or_mode"], row["activities_indices"]), axis=1)

In [ ]:
df_wellbehaved["calculated_activity_utility"] = df_wellbehaved.apply(lambda row: calculate_activity_utility(row["selected plan activities"], row["activity_durations"]), axis=1)

In [ ]:
df_wellbehaved["total_utility"] = df_wellbehaved.apply(lambda row: sum(row["calculated_travel_utility"])+ sum(row["calculated_activity_utility"]), axis=1)

In [ ]:
df_wellbehaved["u_diff"] = df_wellbehaved["selected plan utility"] - df_wellbehaved["total_utility"]

In [ ]:
max(df_wellbehaved["u_diff"])
min(df_wellbehaved["u_diff"])


In [ ]:
plt.plot(df_wellbehaved["u_diff"], np.linspace(1, len(df_wellbehaved), len(df_wellbehaved)),"x")

In [ ]:
plt.plot(df_wellbehaved["total_utility"], df_wellbehaved["selected plan utility"], "x")

In [ ]:
broken = df_wellbehaved[df_wellbehaved["total_utility"]>0]

## Whole plan modal flexibility

This requires the function to choose only the longest leg per trip and use that as a proxy for simplicity. Then compare the plan's set of main modes across plans and compare this to utility change across plans. Completed in old code setup. Skipped for now here. 

## Trip-based modal flexibility

Here we can unpick the utility function and compare utility for a person for a given trip. We can look at either a given trip number (e.g., first trip of the day), the longest (distance) trip, a given trip purpose (e.g., travel to work), or get a composite across the whole day (complex!!). We can also look at just the longest leg of the trip for simplicity or look at the total set of legs together. To do this well, we need to understand the config files properly though - **AGH**.

### First trip of the day, combining all legs

1. Separate out the trips and choose the first (for each plan)
2. Find the change in mode by comparing the whole combination of modes -> check, this might be problematic and we may need to revert to looking at main mode (by distance) for this step
3. Calculate the utility lost for each leg and sum to find utility of trip. For each plan
4. Find the change in utility across plans
5. Find the flexibility by comparing change in mode to change in plans